In [1]:
from flo_ai import Flo
from flo_ai import FloSession
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

# Setting up the send email tool

In [2]:
from typing import Optional, Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class SendEmailInput(BaseModel):
    email: str = Field(description="The email text to be sent")

class SendEmailTool(BaseTool):
    name = "email_triage"
    description = "useful for when you need to send an email to someone"
    args_schema: Type[BaseModel] = SendEmailInput

    def _run(
        self, email: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        print(email)
        return "Email sent successfully"

    async def _arun(
        self, email: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        print(email)
        return "Email sent successfully"

# Setting up the fetch transaction tool

In [3]:
from typing import Optional, Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class FetchTrxInput(BaseModel):
    reference_number: str = Field(description="The transaction reference number")

class FetchTransactionTool(BaseTool):
    name = "fetch_transactions"
    description = "useful for when you want to fetch the transaction details given reference number"
    args_schema: Type[BaseModel] = FetchTrxInput

    def _run(
        self, reference_number: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        return "The transaction happened on 23/07/2024 IST and it failed because there was not enough balance in the account"

    async def _arun(
        self, reference_number: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        return "The transaction happened on 23/07/2024 IST and it failed because there was not enough balance in the account"

# Setup a retriver db

In [4]:
# import
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter

# load the document and split it into chunks
loader = TextLoader("./rag_document.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

/Users/vizsatiz/Documents/hub/flo/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/vizsatiz/Documents/hub/flo/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
from flo_ai.retrievers.flo_retriever import FloRagBuilder
from flo_ai.retrievers.flo_compression_pipeline import FloCompressionPipeline

llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')
session = FloSession(llm)
builder = FloRagBuilder(session, db.as_retriever())
compression_pipeline = FloCompressionPipeline(OpenAIEmbeddings(model="text-embedding-3-small"))
compression_pipeline.add_embedding_reduntant_filter()
compression_pipeline.add_embedding_relevant_filter()
# Reranking

retriever_tool = builder.with_multi_query().with_compression(compression_pipeline).build_retriever_tool(name="HousingLoanRetreiver",
                                                                                               description="Tool to fetch data around housing loans")
session.register_tool(name="HousingLoanTool", tool=retriever_tool)

# Register the tools

In [6]:
session.register_tool(
    name="SendEmailTool", 
    tool=SendEmailTool()
).register_tool(
    name="FetchTransactionTool",
    tool=FetchTransactionTool()
)

# Setting up the agentic flo

In [7]:
agent_yaml = """
apiVersion: flo/alpha-v1
kind: FloRoutedTeam
name: support-email-handler
team:
    name: SupportTicketHandler
    router:
        name: SupportSupervisor
        kind: supervisor
    agents:
      - name: EmailSender
        job: You are capable of sending the reply email but constructing a apt response
        tools:
          - name: SendEmailTool
      - name: TransactionFetcher
        job: You are capable of fetching any kind of transactions from the database given transaction reference id
        tools:
          - name: FetchTransactionTool
      - name: HousingLoanTeamLead
        job: Fetch the housing loan information from the db and answer the question
        tools:
          - name: HousingLoanTool
"""

# Setup input prompt

1. This works if the transaction number is passed
2. Asks for transaction number if its missing

In [8]:
email_input_for_transaction_without_id = """
Hi Tom,

We have a failed transaction. Can you tell me more about why this failed.

Thanks
Vishnu
"""

email_input_for_transaction_with_id = """
Hi Tom,

We have a failed transaction. Can you tell me more about why this failed.

Transaction ID: 12123123432

Thanks
Vishnu
"""

email_input_for_housing_loan = """
Hi Tom,

I am looking for a housing loan, is the interest percentage on the loan

The tenure is 8 years
Amount am looking for is 85 lakhs
And am looking for fixed rate housing loan

Thanks
Vishnu
"""

In [9]:
input_prompt = """
You are a support ticket handling agent. You are expected to understand the email, and perform the following steps:

1. Reply asking for more information if there is missing information in the email
2. Perform the neccessary steps to required to answer the users question and reply the user query as email.
3. If are not able to come up with a answer just reply I dont know over the email

Email:

""" + email_input_for_transaction_with_id

## Run the flo

In [10]:
flo: Flo = Flo.build(session, yaml=agent_yaml)
for s in flo.stream(input_prompt):
     if "__end__" not in s:
        print(s)
        print("----")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'SupportSupervisor-pwZRF': {'next': 'TransactionFetcher-lpv5Y'}}
----


> Entering new AgentExecutor chain...

Invoking: `fetch_transactions` with `{'reference_number': '12123123432'}`


The transaction happened on 23/07/2024 IST and it failed because there was not enough balance in the accountSubject: Re: Failed Transaction Inquiry

Hi Vishnu,

Thank you for reaching out regarding the failed transaction.

The transaction with ID 12123123432 occurred on 23/07/2024 IST and unfortunately failed due to insufficient balance in the account.

If you have any further questions or need assistance with anything else, feel free to ask!

Best regards,  
Tom

> Finished chain.
{'TransactionFetcher-lpv5Y': {'messages': [HumanMessage(content='Subject: Re: Failed Transaction Inquiry\n\nHi Vishnu,\n\nThank you for reaching out regarding the failed transaction.\n\nThe transaction with ID 12123123432 occurred on 23/07/2024 IST and unfortunately failed due to insufficient balance in the account.\n\nIf y